# DG_China
This project is based on the Deep Gavity model, which is based on 20 years of national inter-city foot traffic of Gaode Map as a sample of mobility, with a view to predicting the changes in the attractiveness of cities in the urban agglomerations to the population from 2025.

## 1.Initialization

In [1]:
import pandas as pd
import numpy as np
import sys,torch,time,random,re,os
from sklearn import linear_model

from DGModel import FlowDataset, instantiate_model

db_dir = './data/'                #database_dir

osm_path = db_dir + 'data_osm/'
flows_path = db_dir + 'data_flows/'
population_path = db_dir + 'data_population/'
prediction_path = db_dir + 'data_prediction/'

pt_file_path     = 'result/model_china.pt'
result_file_path = 'result/result_25.csv'
 
city_info = pd.read_csv(db_dir + 'city_info.csv') 
forecast_city_code_list = city_info.dropna(axis=0,how='any')['city_code'].values

In [2]:
batch_size = 1
test_batch_size = 1

device = 'gpu'
seed = 1234
lr = 5e-6
momentum = 0.9
epochs = 60
log_interval = 1

cuda = 1
torch.cuda.manual_seed(seed)
torch_device = torch.device("cuda")

## 2.Training

### 2-1. Function preparation

In [3]:
def train(epoch):
    model.train()
    running_loss = 0.0
    training_acc = 0.0

    for batch_idx, data_temp in enumerate(train_loader):
        b_data = data_temp[0]
        b_target = data_temp[1]
        ids = data_temp[2]
        optimizer.zero_grad()
        loss = 0.0
        for data, target in zip(b_data, b_target):

            if cuda:
                data, target = data.cuda(), target.cuda()

            output = model.forward(data)
            loss += model.loss(output, target)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if batch_idx % log_interval == 0:
            if batch_idx * len(b_data) == len(train_loader) - 1:
                print('Train Epoch: {} [{}/{} \tLoss: {:.6f}'.format(epoch, batch_idx * len(b_data), len(train_loader),
                                                                     running_loss / len(train_dataset)))

def test(epoch):
    model.eval()
    with torch.no_grad():
        test_loss = 0.
        test_accuracy = 0.
        n_origins = 0
        for batch_idx, data_temp in enumerate(test_loader):
            b_data = data_temp[0]
            b_target = data_temp[1]
            ids = data_temp[2]

            for data, target in zip(b_data, b_target):
                if cuda:
                    data, target = data.cuda(), target.cuda()

                output = model.forward(data)
                test_loss += model.loss(output, target).item()

                cpc = model.get_cpc(data, target)
                test_accuracy += cpc
                n_origins += 1

        test_loss /= n_origins
        test_accuracy /= n_origins
        
        return test_accuracy

### 2-2. Data preparation

In [4]:
#[df]features: 9 features, latitude and longitude of each city in the forecast year are needed
features_name = ['transport','traffic','roads','railway','poi','places','landuse','building','population']

features = pd.read_csv(osm_path + 'osm_20_china.csv', engine='python', encoding='utf-8')
population_csv = pd.read_csv(population_path + 'population_20_china.csv', engine='python')

features = pd.merge(features, population_csv.loc[:,['city_code','population']], how='left', on='city_code', copy=False)
features = features.set_index('city_code')

In [5]:
# [dict]oa2features    
oa2features = dict()
for oa in city_info['city_code']:
    oa_features = list(features.loc[oa,features_name].values)
    oa_features = list(np.log(oa_features))
    oa2features[oa] = oa_features

In [6]:
# [dict]o2d2flow  
od = pd.read_csv(flows_path + 'flows_20.csv')
od = od.groupby(['from_city_code','to_city_code'])['real_value'].sum()
o2d2flow = {}
for (o,d),f in od.items():      #Convert tabel to dictionary
    try:
        d2f = o2d2flow[o]
        d2f[d] = f
    except KeyError:
        o2d2flow[o] = {d: f}

In [7]:
# [dict]tileid2oa2features2vals
tileid2oa2features2vals = {}
for _,(tile_num,city_num) in city_info.loc[:,['tile_num','city_code']].iterrows():
    city_features = features.loc[city_num, features_name].to_dict()
    try:
        reg = tileid2oa2features2vals[tile_num]
        reg[city_num] = city_features
    except KeyError:
        tileid2oa2features2vals[tile_num] = {city_num:city_features}

In [8]:
# [dict]oa2centroid 
oa2centroid = {key: [float(features['latitude'][key]), float(features['longitude'][key])]  for key in features.index}

In [9]:
# [list]train_data,test_data: selected the tile_id for train and test

random_index = np.load(db_dir + 'random_index.npy')
half_index = int(len(random_index)/2)

train_data_index = random_index[0:half_index]     #tileid for train
test_data_index = random_index[half_index+1:]     #tileid for test
tileid_keys = list(tileid2oa2features2vals.keys())
train_data = [m for t in train_data_index for m in list(tileid2oa2features2vals[tileid_keys[t]].keys())]
test_data = [m for t in test_data_index for m in list(tileid2oa2features2vals[tileid_keys[t]].keys())]

In [10]:
train_dataset_args = {'tileid2oa2features2vals': tileid2oa2features2vals,\
                      'o2d2flow': o2d2flow,\
                      'oa2features': oa2features,\
                      'oa2centroid': oa2centroid,\
                      'dim_dests': 512,\
                      'frac_true_dest': 0.0}
test_dataset_args = {'tileid2oa2features2vals': tileid2oa2features2vals,\
                     'o2d2flow': o2d2flow,\
                     'oa2features': oa2features,\
                     'oa2centroid': oa2centroid,\
                     'dim_dests': int(1e9),\
                     'frac_true_dest': 0.0}

train_dataset = FlowDataset(train_data, **train_dataset_args)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

test_dataset = FlowDataset(test_data, **test_dataset_args)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size)   #让数据做核酸排好队

dim_input = len(train_dataset.get_features(train_data[0], train_data[0]))          #equal 19

### 3-2. Training Cycle

In [11]:
model = instantiate_model(oa2centroid, oa2features, dim_input, device=torch_device)
if device.find("gpu") != -1:
    model.cuda()

optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, momentum=momentum)

t0 = time.time()
test_accuracy = test(0)
best_test_accuracy = 0.0

for epoch in range(1, epochs + 1):
    # set new random seeds
    torch.manual_seed(seed + epoch)
    np.random.seed(seed + epoch)
    random.seed(seed + epoch)

    train(epoch)
    test_accuracy = test(epoch)
    print(test_accuracy)

    if best_test_accuracy < test_accuracy:
        best_test_accuracy = test_accuracy

        print('Saving model to {} ...'.format(pt_file_path))
        torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict()}, pt_file_path)

t1 = time.time()
print("Total training time: %s seconds" % (t1 - t0))
print('Computing the CPC on test set, loc2cpc_numerator ...')

Train Epoch: 1 [163/164 	Loss: 124.872150
0.5673759373961054
Saving model to result/model_china.pt ...
Train Epoch: 2 [163/164 	Loss: 113.784211
0.6257968718872133
Saving model to result/model_china.pt ...
Total training time: 7.279669761657715 seconds
Computing the CPC on test set, loc2cpc_numerator ...


## 3.Prediction

### 3-1. Function preparation

In [12]:
def generate():
    
    result = list()
    
    model.eval()
    with torch.no_grad():
        
        for batch_idx, data_temp in enumerate(generate_loader):
            b_data = data_temp[0]
            from_city_code = data_temp[2][0].tolist()[0]
            to_city_list = data_temp[3][0].tolist()[0]

            for data in b_data:
                if cuda:
                    data = data.cuda()

                output = model.forward(data)
                soft = torch.nn.functional.softmax(output,dim=1).reshape(1,-1)
                soft = soft[0].data.tolist()
                
                for index in range(len(to_city_list)):
                    result.append([int(from_city_code), int(to_city_list[index]), soft[index]])
        
        return pd.DataFrame(result, columns=['from_city_code', 'to_city_code', 'forecast'],dtype=str)

### 3-2. Data preparation for prediction

In [13]:
# [df]features: 9 features, latitude and longitude of each city in the forecast year are needed
features_name = ['transport','traffic','roads','railway','poi','places','landuse','building','population']

features = pd.read_csv(prediction_path + '/osm_25.csv', engine='python', encoding='utf-8')
population_csv = pd.read_csv(population_path + 'population_25_agglomerate.csv', engine='python',encoding='utf-8')

features = pd.merge(features, population_csv.loc[:,['city_code','population']], how='left', on='city_code', copy=False)
features = pd.merge(features, city_info.loc[:,['city_code','longitude','latitude']], how='left', on='city_code', copy=False)
features= features.set_index('city_code')

city_info_agglomerate = city_info.dropna(axis=0,how='any')

In [14]:
# [dict]oa2features
oa2features = dict()
for oa in city_info_agglomerate['city_code']:
    oa_features = list(features.loc[oa,features_name].values)
    oa_features = list(np.log(oa_features))
    oa2features[oa] = oa_features

In [15]:
# [dict]o2d2flow
o2d2flow = dict()

In [16]:
# [dict]tileid2oa2features2vals
tileid2oa2features2vals = {}
for _,(tile_num,city_num) in city_info_agglomerate.loc[:,['agglomerate','city_code']].iterrows():
    city_features = features.loc[city_num, features_name].to_dict()
    try:
        reg = tileid2oa2features2vals[tile_num]
        reg[city_num] = city_features
    except KeyError:
        tileid2oa2features2vals[tile_num] = {city_num:city_features}
        
# [dict]oa2centroid 
oa2centroid = {key: [float(features['latitude'][key]), float(features['longitude'][key])]  for key in features.index}

generate_data = list(features.index)

In [17]:
generate_dataset_args = {'tileid2oa2features2vals': tileid2oa2features2vals,\
                      'o2d2flow': o2d2flow,\
                      'oa2features': oa2features,\
                      'oa2centroid': oa2centroid,\
                      'dim_dests': 512,\
                      'frac_true_dest': 0.0}

generate_dataset = FlowDataset(generate_data, **generate_dataset_args)
generate_loader = torch.utils.data.DataLoader(generate_dataset, batch_size=test_batch_size)

dim_input = len(generate_dataset.get_features(generate_data[0], generate_data[0])) 

### 3-3. Prediction

In [18]:
model = instantiate_model(oa2centroid, oa2features, dim_input, device=torch_device)
if device.find("gpu") != -1:
    model.cuda()

optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, momentum=momentum)

checkpoint = torch.load(pt_file_path)                                                #load the trained model
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

result = generate()

### 3-4. Data Export

In [19]:
result = pd.merge(result,city_info.astype(str).loc[:,['city_code','longitude','latitude','city']],left_on='from_city_code',right_on='city_code',copy=False)
result = result.drop('city_code',axis=1).rename(columns={'longitude':'from_city_longitude','latitude':'from_city_latitude','city':'from_city_name'})
result = pd.merge(result,city_info.astype(str).loc[:,['city_code','longitude','latitude','city']],left_on='to_city_code',right_on='city_code',copy=False)
result = result.drop('city_code',axis=1).rename(columns={'longitude':'to_city_longitude','latitude':'to_city_latitude','city':'to_city_name'})

result.to_csv(result_file_path)